In [1]:
# !pip install deepface
# !pip install cv2
# !pip install matplotlib
# !pip install keyboard
# !pip install pygame
# !pip install datetime
# !pip install xlwings

In [2]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt

In [14]:
import numpy, datetime, pygame, keyboard
import xlwings as xw

pygame 2.5.1 (SDL 2.28.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [16]:
worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [17]:
s = 2

i = 0

entered = False
al_entered = False

In [18]:
cap = cv2.VideoCapture(0)

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False

    state, frame = cap.read()

    if state != True:
        break

    res = DeepFace.find(frame, db_path='deepface/database/', enforce_detection=False, model_name='VGG-Face')

    if len(res[0]['identity']) > 0:
        
        name = res[0]['identity'][0].split('/')[2]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)

        cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
        cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
        cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)

        n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
        n_txtrect = n_txt.get_rect()
        n_txtrect.center =(960-200, 540//2)
        
        d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
        d_txtrect = d_txt.get_rect()
        d_txtrect.center= (960-200, (540//2)+40)
        
        t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
        t_txtrect = d_txt.get_rect()
        t_txtrect.center = (960-200, (540//2)+80)         
        
        screen.blit(n_txt, n_txtrect)
        screen.blit(d_txt, d_txtrect)
        screen.blit(t_txt, t_txtrect)

        if keyboard.is_pressed('enter'):
                
            if name not in students:
                
                worksheet.range(f'A{s}').value = name
                worksheet.range(f'B{s}').value = date
                worksheet.range(f'c{s}').value = time
                students.append(name)
                s+=1
                entered =True
            else:
                al_entered = True
                    
        if i<30 and entered==True:
            e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
            e_txtrect = e_txt.get_rect()
            e_txtrect.center = (960-200, (540//2)+120)
            screen.blit(e_txt, e_txtrect)
            i+=1

        elif i<30 and al_entered==True:
            ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
            ae_txtrect = ae_txt.get_rect()
            ae_txtrect.center = (960-200, (540//2)+120)
            screen.blit(ae_txt, e_txtrect)    
            i+=1

        else:
            entered=False
            al_entered = False
            i = 0

    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
# cv2.destroyAllWindows()

There are  40  representations found in  representations_vgg_face.pkl
find function lasts  9.10754656791687  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.8699979782104492  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.8390035629272461  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.8349514007568359  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.834050178527832  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.8930063247680664  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  0.942556619644165  seconds
There are  40  representations found in  representations_vgg_face.pkl
find function lasts  1.05800461769104  seconds
There are  40  representations found in  representatio

In [3]:
# model = DeepFace.find(img_path = 'deepface/test/natascha_test.jpg', db_path = 'deepface/database/', model_name = 'VGG-Face', enforce_detection = False)

In [4]:
# model[0]

In [5]:
# model[0]['identity'][0].split('/')[2]

In [6]:
# def label_face(pred_path, training_file):

#     model = DeepFace.find(img_path = pred_path, db_path = training_file, model_name = 'VGG-Face', enforce_detection = False)

#     pred_label = model[0]['identity'][0].split('/')[2]

#     img = cv2.imread(pred_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     x = model[0]['source_x'][0]
#     y = model[0]['source_y'][0]
#     w = model[0]['source_w'][0]
#     h = model[0]['source_h'][0]
#     img2 = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 225), 10)
#     img2 = cv2.putText(img, pred_label, (x, y - 100), cv2.FONT_HERSHEY_COMPLEX, 6, (0, 225, 0), 4, cv2.LINE_AA)
#     plt.imshow(img2)
#     plt.axis("off");

In [7]:
# label_face('deepface/test/nihal_test.jpg', 'deepface/database/')

In [8]:
# label_face('deepface/test/jeremy_test.jpg', 'deepface/database/')

In [9]:
# label_face('deepface/test/riri_test.jpg', 'deepface/database/')

In [10]:
# label_face('deepface/test/jeevan_test.jpg', 'deepface/database/')

In [11]:
# model = DeepFace.find(img_path = 'deepface/database/jeevan/jeevan3.jpg', db_path = 'deepface/database/', model_name = 'VGG-Face', enforce_detection = False)

In [12]:
# model[0]

In [13]:
# label_face('deepface/database/jeevan/jeevan5.jpg', 'deepface/database/')